### Overview ###

This notebook is divided into the following section:
- TakeCV corpus
    - Pre-annotation Preprocessing
    - Pos-annotation Preprocessing
- Survey corpus
    - Preprocessing
    
**TakeCV corpus** is a sub-corpus, a part of the ***PentoRef*** (Zarrieß et al., 2016). It consists of monologues of Human agent describing verbally a pentomino piece in German to a wizard, which has to correctly identify the piece in the real-world scene.
- In the pre-annotation step, we removed the non-verbal transcription and translate the verbal description to English with DeepL.
- In the annotation step, we manually identify the labels of each pentomino piece, along with the color, shape, position, scene ID, and recheck the machine translated utterance. After that, we do a short data survey on label.

**Survey corpus** is our own data, collected from a closed survey. Participants were asked to type down the description of a pentomino piece they saw in the real-world scene. Here, we also clean the data and do a short data survey.

# TakeCV corpus
## Pre-annotation Preprocessing

In [ ]:
import numpy as np
import pandas as pd
import os
import re

In [ ]:
# https://pypi.org/project/deepl/
#pip install deepl
import deepl

In [ ]:
# need authen key from free deepl api account # up to 500k char/month
auth_key = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx:xx"  
translator = deepl.Translator(auth_key)

In [5]:
path = os.getcwd()
xlsx_files = os.listdir(path+'\\raw\\')
xlsx_files # importing from excel allows vowels with umlaut, but csv could not read them

['takecv_r1.xlsx',
 'takecv_r2.xlsx',
 'takecv_r3.xlsx',
 'takecv_r4.xlsx',
 'takecv_r5.xlsx',
 'takecv_r6.xlsx',
 'takecv_r7.xlsx',
 'takecv_r9.xlsx']

In [1]:
# exploring data from one of the file

# importing from excel allows vowels with umlauts, but csv could not read them
df = pd.read_excel('raw/takecv_r5.xlsx').iloc[:, 0].str.split(',', expand=True)

**Preprocessing steps**

* everything inside <v="..."> should replace the one in <v="...">...\</v>
* if between \<p> and \</p> then extract it
* everything else inside <...> <.../> </...> {...} can be deleted
* clean out anything which isn't A-Za-z0-9()-+
* when two utterances belong to the same ( + ), deal with them in the later stage
* translate to English

In [7]:
def preproc_df(df):

    for row in df.iterrows():
        
        text = row[1][3]
        t1 = re.sub('<v=""(.+?)"">(.+?)</v>',r'\1', text) # get the right word from the spoken variants
        t2 = re.sub('<p>(.+?)</p>',r'\1', t1) # get rid of marks in partial words
        t3 = re.sub(r"\{[^()]*\}", "", t2) # get rid of fillers
        t4 = re.sub(r"\<[^()]*\>", "", t3) # get rid of nonverbal markers
        t5 = re.sub(r'[^a-zA-Z0-9äöüÄÖÜß\(\)\-\+\.\s]', '', t4) # keep only these letters
        if len(t5) > 0:
            t6 = translator.translate_text(t5, target_lang="EN-US") # translate with deepL
        else: t6 = None
    
        row[1][4] = t5
        row[1][5] = t6
        
    
    return df

In [ ]:
for f in xlsx_files:
    # read file
    df = pd.read_excel('raw/'+f).iloc[:, 0].str.split(',', expand=True)
    
    # add new columns for cleaned texts and translation
    df[4] = df.apply(lambda _: '', axis=1)
    df[5] = df.apply(lambda _: '', axis=1)
    
    #preprocessing
    df_clean = preproc_df(df)
    
    #save to files
    df_clean
    df_clean.drop([6,7], axis=1).to_excel('takecv'+f[-7:-5]+'_processed.xlsx', index=False)

## Text Processing

We take the 8 annotated files from the folder 'Translation' and combine them into one. Then, we process the text into ['label (pentomino piece's color and shape)','text (piece description)']

In [2]:
path = os.getcwd()
xlsx_files = os.listdir(path+'\\annot_2706\\')
xlsx_files

['takecvr1_processed.xlsx',
 'takecvr2_processed.xlsx',
 'takecvr3_processed.xlsx',
 'takecvr4_processed.xlsx',
 'takecvr5_processed.xlsx',
 'takecvr6_processed.xlsx',
 'takecvr7_processed.xlsx',
 'takecvr9_processed.xlsx']

In [3]:
df = pd.read_excel('annot_2706/takecvr1_processed.xlsx')
print ('takecvr1_processed.xlsx done')

for file in xlsx_files[1:]:
    read = pd.read_excel('annot_2706/'+file)
    df = pd.concat([df, read])
    print(file, 'done')

takecvr1_processed.xlsx done
takecvr2_processed.xlsx done
takecvr3_processed.xlsx done
takecvr4_processed.xlsx done
takecvr5_processed.xlsx done
takecvr6_processed.xlsx done
takecvr7_processed.xlsx done
takecvr9_processed.xlsx done


In [5]:
df = df.rename(columns={5:'translate', 6:'correct'}).drop(columns=[0,1,2,3,4,'shade','Unnamed: 17'])
# r1, r2, r3's idx ranked within the scene

In [6]:
#check missing values
df.isna().sum()

translate     228
correct      1549
color        1168
shape        1169
up           1169
down         1170
left         1169
right        1169
middle       1169
scene        1169
idx          1169
dtype: int64

In [ ]:
# if no translation, then filter out the row
df_filt = df[~df['translate'].isna()] # filter out when no translation or translation correction

# filter out lines which are not object description
df_filt = df_filt[df_filt['correct']!='x']

# if the correct column is empty, then take the value from the translate column
df_filt['correct'].fillna(df_filt['translate'], inplace=True) # copy values from translate to correct if correct is empty

# fill the cell with the value in the cell above if cell is empty
df_fill = df_filt.fillna(method='ffill')

In [12]:
# check missing values
df_fill.isna().sum()

translate    0
correct      0
color        0
shape        0
up           0
down         0
left         0
right        0
middle       0
scene        0
idx          0
dtype: int64

In [13]:
# clean color and shape columns
df_fill['color'] = df_fill['color'].replace(r'[^a-z]', "", regex=True)
df_fill['shape'] = df_fill['shape'].replace(r'[^a-z]', "", regex=True)

In [14]:
# check value in each column
for col in df_fill.columns[2:-1]:
    print (df_fill[[col]].value_counts())

color 
wooden    453
blue      230
green     153
orange    134
yellow    103
purple     96
gray       88
pink       67
red        61
dtype: int64
shape
f        148
l        126
w        126
y        124
t        119
v        117
z        117
n        113
i        112
u        106
p        101
x         76
dtype: int64
up 
0.0    951
1.0    434
dtype: int64
down
0.0     966
1.0     419
dtype: int64
left
0.0     942
1.0     443
dtype: int64
right
0.0      908
1.0      477
dtype: int64
middle
0.0       789
1.0       596
dtype: int64
scene
r3_2     80
r3_1     76
r1_1     70
r1_2     64
r4_3     57
r2_2     57
r4_1     54
r3_3     53
r7_3     49
r2_1     44
r7_4     42
r7_2     41
r4_4     36
r6_4     34
r6_2     33
r7_1     32
r7_6     31
r6_7     30
r7_5     30
r5_6     30
r6_6     28
r4_2     28
r9_3     25
r6_3     25
r5_4     25
r6_5     25
r5_2     25
r5_5     25
r5_7     24
r5_3     23
r5_1     23
r9_6     21
r6_1     21
r5_8     20
r9_5     19
r9_2     17
r2_3     17
r9_1     16
r

In [15]:
df_fill.to_csv('takeCV_concat_2706.csv')
print (df_fill.shape)

(1385, 11)


In [3]:
# check lines with translation correction
df_fill[df_fill['correct']!=df_fill['translate']]

In [17]:
df_fill['label'] = df_fill['color'] + ' ' + df_fill['shape']

In [18]:
# check if any label does not exist
target = ['wooden v','blue v','purple n','wooden n','blue z','wooden z',
          'green t','yellow t','green w','wooden w','blue i','wooden i',
          'yellow u','orange u','pink p','wooden p','red x','wooden x',
          'wooden y','gray f','wooden f','wooden l','orange l']

# looking for wrongly labeled data
to_fix = df_fill[~df_fill['label'].isin(target)]

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

# print results if found wrong labels
print (to_fix[['scene','label','idx']].drop_duplicates())

In [20]:
print (df_fill.shape)

df_cat = df_fill.groupby(['label', 'color', 'shape', 'up', 'down', 
                          'left', 'right', 'middle', 'scene', 'idx'], as_index = False).agg({'correct': ' '.join})

#clean correct #punc
#clean color/shape #non a-z

print (df_cat.shape)

(1385, 12)
(525, 11)


In [21]:
df_cat['label'].value_counts()

wooden y    39
green t     36
gray f      33
green w     32
orange l    32
blue i      30
red x       30
pink p      30
blue z      29
yellow u    28
purple n    28
blue v      25
yellow t    20
wooden v    19
orange u    19
wooden f    17
wooden w    15
wooden i    14
wooden z    13
wooden p    12
wooden l    11
wooden n    8 
wooden x    5 
Name: label, dtype: int64

In [23]:
text_list = []
label_list = []
data_list = []

for row in df_cat.iterrows():
    label = row[1][0]
    text = row[1][-1]
    
    c_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    c_text = re.sub(r' +',' ', c_text.strip().lstrip())
    
    #print (c_color, c_shape, c_text)
    
    text_list.append(c_text)
    label_list.append(label)
    data_list.append([label, c_text])

In [4]:
#data_list
#data_df = pd.DataFrame(text_list,label_list).reset_index().rename(columns={'index':'label', 0: 'text'})
#data_df

# Survey Corpus
## Preprocessing

In [9]:
data = pd.read_excel('survey_full.xlsx', sheet_name='label')

In [14]:
# lowercase and get rid of punctuation marks (except - and \s)
df = data.apply(lambda x: x.astype(str).str.lower()).apply(lambda x: x.astype(str).str.replace('[^a-zA-Z\-\s]', ''))

In [15]:
# get rid of extra space
df['label'] = df['label'].apply(lambda x: re.sub(r' +',' ', x.strip().lstrip()))
df['text'] = df['text'].apply(lambda x: re.sub(r' +',' ', x.strip().lstrip()))

In [6]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)
#print (df)

In [17]:
# count label
single = df[df['task'] == 'single']
scene = df[df['task'] == 'scene']

print (single[['label']].value_counts())
print (scene[['label']].value_counts())

label   
blue i      8
blue z      8
gray f      8
green t     8
green w     8
orange u    8
pink p      8
purple n    8
red x       8
wooden l    8
wooden v    8
wooden y    8
dtype: int64
label   
blue v      8
orange l    8
orange t    8
wooden f    8
wooden i    8
wooden p    8
wooden w    8
wooden x    8
wooden y    8
yellow u    8
dtype: int64


In [64]:
df.to_excel('processed_survey.xlsx', index=False)